In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')

sys.path.append("/content/drive/My Drive/Colab Notebooks/myfs")

import data_lo
import fsrcnn

def main():
    folder = "/content/drive/My Drive/Colab Notebooks/tens/00000"
    scale = 3
    hr_size = (768, 768)
    lr_size = (hr_size[0] // scale, hr_size[1] // scale)

    lr_images, hr_images = data_lo.load_dataset_from_folder(folder, scale=scale, hr_size=hr_size)
    print(f"Loaded {len(lr_images)} LR-HR image pairs")

    lr_images = np.array(lr_images)
    hr_images = np.array(hr_images)

    print(f"LR images shape: {lr_images.shape}")
    print(f"HR images shape: {hr_images.shape}")

    lr_train, lr_val, hr_train, hr_val = train_test_split(lr_images, hr_images, test_size=0.1, random_state=42)

    # model = fsrcnn.fsrcnn_model(input_shape=(lr_size[0], lr_size[1], 3), scale=scale)
    # model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    # model.summary()



    model_path = "/content/drive/My Drive/Colab Notebooks/Super_resolution_SRCNN_final.h5"
    model = tf.keras.models.load_model(model_path, compile=False)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    model.summary()

    checkpoint_path = "/content/sr_cnn_model.h5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True, monitor="val_loss", mode="min")

    history = model.fit(
        lr_train,
        hr_train,
        validation_data=(lr_val, hr_val),
        epochs=50,
        batch_size=4,
        callbacks=[checkpoint]
    )

    plt.figure(figsize=(8,5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title("Training and Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("MSE Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

    final_model_path = "/content/drive/My Drive/Colab Notebooks/Super_resolution_SRCNN_final.h5"
    model.save(final_model_path)
    print(f"Model saved to {final_model_path}")

if __name__ == "__main__":
    main()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install necessary packages
!pip install numpy
!pip install tensorflow
!pip install opencv-python
!pip install gdown --quiet
!pip install -q PyDrive2 google-auth-oauthlib

In [ ]:
#to load dataset from public Google Drive folder 

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

folder_id = "1taHKxS66YKJNhdhiGcEdM6nnE5W9zBb1"  # your folder ID
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

In [ ]:

import os
os.makedirs('data', exist_ok=True)

for file in file_list:
    fname = file['title']
    print("Downloading", fname)
    file.GetContentFile(os.path.join('data', fname))